In [321]:
import requests
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Boolean, SmallInteger, CheckConstraint, Float
from sqlalchemy.orm import sessionmaker


In [322]:
liste_mois = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
liste_annee = ["2016", "2017", "2018", "2019", "2020", "2021"]

In [323]:
liste_response = []
for annee  in liste_annee:
    for mois in liste_mois:
        url = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&q=&rows=10000&sort=date&facet=date&facet=gc_obo_date_heure_restitution_c&facet=gc_obo_gare_origine_r_name&facet=gc_obo_nature_c&facet=gc_obo_type_c&facet=gc_obo_nom_recordtype_sc_c&refine.date={}%2F{}".format(annee, mois)
        response = requests.get(url)
        liste_response.append(response)

In [324]:
response = requests.get(url)
response.json()


{'nhits': 6044,
 'parameters': {'dataset': 'objets-trouves-restitution',
  'rows': 10000,
  'start': 0,
  'sort': ['date'],
  'facet': ['date',
   'gc_obo_date_heure_restitution_c',
   'gc_obo_gare_origine_r_name',
   'gc_obo_nature_c',
   'gc_obo_type_c',
   'gc_obo_nom_recordtype_sc_c'],
  'format': 'json',
  'timezone': 'UTC'},
 'records': [{'datasetid': 'objets-trouves-restitution',
   'recordid': '559884fe3255a1743a41312f4228d6965ab2b5d3',
   'fields': {'gc_obo_gare_origine_r_code_uic_c': '0087286542',
    'gc_obo_date_heure_restitution_c': '2022-01-05T15:26:46+00:00',
    'gc_obo_type_c': 'Bagagerie: sacs, valises, cartables',
    'gc_obo_gare_origine_r_name': 'Tourcoing',
    'gc_obo_nature_c': 'Valise, sac sur roulettes',
    'gc_obo_nom_recordtype_sc_c': 'Objet trouvé',
    'date': '2021-12-31T23:24:46+00:00'},
   'record_timestamp': '2022-12-21T06:00:57.413Z'},
  {'datasetid': 'objets-trouves-restitution',
   'recordid': '8107cd28f611cb7d2c027597e94954ed761ea1c2',
   'fields'

In [325]:
engine = create_engine('sqlite:///sncf_table.sqlite')
Base = declarative_base()

class Gare(Base):
    __tablename__ = "Gare"

    id = Column(String, primary_key = True)
    code_uic = Column(String, nullable=True)
    nom_gare = Column(String, nullable=True)
    code_postal = Column(String, nullable=True)
    code_commune = Column(String, nullable=True)
    code_departement = Column(String, nullable=True)
    departement = Column(String, nullable=True)
    longitude = Column(Float, nullable=True)
    latitude = Column(Float, nullable=True)


class Lost_Item(Base):
    __tablename__ = "Lost_Item"

    id = Column(String, primary_key=True, nullable=False)
    date = Column(String)
    type_objets = Column(String)
    gare= Column(String)
    code_uic = Column(String)
    date_heure_restitution = Column(String)

class Frequentation(Base) :
    __tablename__ = "Frequentation"
    
    id = Column(String, primary_key=True)
    code_uic = Column(String, nullable = True)
    code_postal = Column(String, nullable=True)
    total_voyageurs_2016 = Column(String, nullable=True)
    total_voyageurs_2017 = Column(String, nullable=True)
    total_voyageurs_2018 = Column(String, nullable=True)
    total_voyageurs_2019 = Column(String, nullable=True)
    total_voyageurs_2020 = Column(String, nullable=True)
    total_voyageurs_2021 = Column(String, nullable=True)

Base.metadata.create_all(engine)

## Données objets

In [326]:
Session = sessionmaker(bind=engine)
session = Session()

dico = {'date': 'date_value', 'gc_obo_type_c': 'value1', 'gc_obo_gare_origine_r_name': 'value2', 'gc_obo_gare_origine_r_code_uic_c': 'value3', 
'gc_obo_date_heure_restitution_c': 'value4'}

for response in liste_response:
    for itemlost in response.json()["records"]:
        for cle in ['date','gc_obo_type_c','gc_obo_gare_origine_r_name','gc_obo_gare_origine_r_code_uic_c','gc_obo_date_heure_restitution_c']:
            try:
                dico[cle] = itemlost['fields'][cle]
            except KeyError:
                dico[cle] = None
        session.add(Lost_Item(id=itemlost['recordid'],
                            date=dico['date'],
                            type_objets=dico['gc_obo_type_c'],
                            gare=dico['gc_obo_gare_origine_r_name'],
                            code_uic=dico['gc_obo_gare_origine_r_code_uic_c'],
                            date_heure_restitution=dico['gc_obo_date_heure_restitution_c']))

session.commit() 

## Données gares

In [ ]:
engine = create_engine('sqlite:///sncf_table.sqlite')
Session = sessionmaker(bind=engine)
session = Session()

url_game = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=referentiel-gares-voyageurs&q=&rows=10000"
my_request = requests.get(url_game)

field_list = [
    ["code_uic", "uic_code"],
    ["nom_gare", "gare_alias_libelle_noncontraint"],
    ["code_commune", "commune_code"],
    ["code_postal", "adresse_cp"],
    ["code_departement", "departement_numero"],
    ["departement", "departement_libellemin"],
    ["longitude", "longitude_entreeprincipale_wgs84"],
    ["latitude", "latitude_entreeprincipale_wgs84"]
]

for gare in my_request.json()["records"]:
    gare_data= {}

    gare_data["id"] = gare["recordid"]

    for field in field_list:
        try: 
            gare_data[field[0]] = gare["fields"][field[1]]
        except KeyError:
            gare_data[field[0]]=None

   
    session.add(Gare(**gare_data))
    session.commit()
    # try:
    #     session.commit()
    # except:
    #     print(gare_data["code_uic"])

## Données frequentation

In [ ]:
engine = create_engine('sqlite:///sncf_table.sqlite')
Session = sessionmaker(bind=engine)
session = Session()

url_game = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=frequentation-gares&q=&sort=nom_gare&rows=10000"
my_request = requests.get(url_game)


field_list = [
    ["code_uic", "code_uic_complet"],
    ["code_postal", "code_postal"],
    ["total_voyageurs_2016", "total_voyageurs_2016"],
    ["total_voyageurs_2017", "totalvoyageurs2017"],
    ["total_voyageurs_2018", "total_voyageurs_2018"],
    ["total_voyageurs_2019", "total_voyageurs_2019"],
    ["total_voyageurs_2020", "total_voyageurs_2020"],
    ["total_voyageurs_2021", "total_voyageurs_2021"]
]

for frequentation in my_request.json()["records"]:
    frequentation_data= {}

    frequentation_data["id"] = frequentation["recordid"]

    for field in field_list:
        try: 
            frequentation_data[field[0]] = frequentation["fields"][field[1]]
        except KeyError:
            frequentation_data[field[0]]=None

   
    session.add(Frequentation(**frequentation_data))
    session.commit()